In [ ]:
# 系统库
import os
import subprocess
import time
import shutil
import json
import socket
from datetime import datetime, timedelta
# 第三方科学计算 & 可视化
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 设置中文字体，避免乱码
plt.rcParams['font.sans-serif'] = ['SimHei']        # 黑体
plt.rcParams['axes.unicode_minus'] = False          # 解决负号显示成方块的问题

# 机器学习 & 优化
import xgboost as xgb
import lightgbm as lgb
import optuna
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer


# Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi




In [ ]:
# 初始化


host = "10.162.147.95"
user = "user1"
password = "123456"

database_name = 'predicting_road_accident_risk'  # 数据库名称
competition = database_name  # 竞赛名称
kaggle_competition_name = "playground-series-s5e10"
study_save_name = "LightGBM_model"





if socket.gethostname() == 'hao-2':
    dir = rf'D:\数据\Kaggle_\{competition}'
else:
    dir = os.getcwd()



DIRS = {
    "dir":              dir,                                       
    "DATA_DIR000":      os.path.join(dir, "DATA_DIR000"),
    "HISTORY":          os.path.join(dir, "HISTORY", f"{study_save_name}"),
    "SUBMISSION":       os.path.join(dir, "SUBMISSION", f"{study_save_name}"),
}

# 自动创建目录
for key, path in DIRS.items():
    os.makedirs(path, exist_ok=True)

# 打印时一行一个地址
print("✅ 路径已创建：\n")
for key, path in DIRS.items():
    print(f"{key:<12} : {path}")


# 数据提取处理

In [ ]:
# 加载Kaggle 训练集和 Bradley 熔点公开数据集

# Kaggle 提供的训练集和测试集
train_df = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "train.csv"))
test_df  = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "test.csv"))



# 输出数据集规模，确认加载成功
print("Train                        shape:", train_df.shape)
print("Test                         shape:", test_df.shape)

In [ ]:
# 检测 DataFrame 每列的数据类型、缺失值情况、唯一值数量，并给出部分示例值。
def dataframe_info(df: pd.DataFrame, n_sample: int = 6) -> pd.DataFrame:
    """
    检测 DataFrame 每列的数据类型、缺失值情况、唯一值数量，并给出部分示例值。
    
    参数:
        df       : 输入的 pandas DataFrame
        n_sample : 每列展示的样本值数量 (默认 3)
    
    返回:
        summary_df : 包含每列信息的 DataFrame
    """
    summary = pd.DataFrame({
        "数据类型": df.dtypes,
        "缺失值数量": df.isnull().sum(),
        "缺失值比例": df.isnull().mean(),
        "唯一值数量": df.nunique()
    })
    
    # 添加示例值（前 n_sample 个唯一值）
    summary["示例值 (samples)"] = df.apply(
        lambda col: col.dropna().unique()[:n_sample] if col.notnull().any() else []
    )
    
    # 按缺失比例排序
    summary = summary.sort_values("缺失值比例", ascending=False)
    print(df.columns.values)
    display(summary)
    # print(summary)


# 使用方法
dataframe_info(train_df, n_sample = 6)
dataframe_info(test_df, n_sample = 6)


# 数据分析

In [ ]:
# 打印清单
def config_to_str(config: dict, indent: int = 0) -> str:
    """递归生成配置字符串"""
    prefix = "     " * indent
    lines = []
    for key, value in config.items():
        if isinstance(value, dict):
            lines.append(f"{prefix}🔹 {key}:")
            lines.append(config_to_str(value, indent + 1))  # 递归拼接子字典
        else:
            lines.append(f"{prefix}- {key:<20}: {value}")
    return "\n".join(lines)



In [ ]:
# 实验配置单
config = {
    # 固定开关
    "ISTEST"            : False,

    "use_feature_gen"   : True,
    "use_pca"           : True,
    "pca_components"    : 10,



    "study_save_name"    : study_save_name,


    
    # 特征选择 XGBoost 参数
    "xgb_selector_model_params": {
        "n_estimators"  : 500,
        "max_depth"     : 6,
        "learning_rate" : 0.05,
        "random_state"  : 2025,
        "device"        : "cpu",
        "objective"     : "reg:squarederror",
        "tree_method"   : "hist",
        "verbosity"     : 0
    },

    "selector_threshold"  : "0*mean",   

    # # 训练设置
    # "xgb_train_model_params": {
    #     'max_depth'   : 6,
    #     'eta'         : 0.1,
    #     'tree_method' : 'hist',
    #     'eval_metric' : 'rmse',
    # },
    # "num_boost_round": 15000,


    # 训练设置（LightGBM）
    "lgb_train_model_params": {
        "objective": "regression",        # 回归任务
        "metric": "rmse",                 # 使用 RMSE 作为评估指标
        "boosting_type": "gbdt",          # 梯度提升树（默认）
        "learning_rate": 0.05,            # 学习率（原来 XGB 的 eta）
        "num_leaves": 31,                 # 控制树的复杂度
        "max_depth": -1,                  # -1 表示不限制深度
        "min_data_in_leaf": 20,           # 每个叶子最少样本数
        "feature_fraction": 0.9,          # 每次建树时使用的特征比例
        "bagging_fraction": 0.8,          # 每次建树时使用的数据比例
        "bagging_freq": 5,                # 每 5 次迭代进行一次 bagging
        "lambda_l1": 0.1,                 # L1 正则化
        "lambda_l2": 0.1,                 # L2 正则化
        "verbosity": -1                   # 不输出详细日志
    },
    "num_boost_round": 15000,             # 与 XGB 相同，用于控制最大迭代次数




}

In [ ]:
# 数据拆分 (特征矩阵 与 目标向量)
# ============================================

import numpy as np
import pandas as pd

def prepare_features_and_target(train_df: pd.DataFrame, test_df: pd.DataFrame, config: dict):
    """
    数据拆分函数：构造训练集和测试集的特征矩阵与目标向量
    """

    
    # 2. 构造特征矩阵和目标向量
    features_train = train_df.drop(columns=['id', 'accident_risk'])   # 训练集特征 (X)
    target_train   = train_df['accident_risk']                            # 训练集目标 (y, 熔点)
    features_test  = test_df.drop(columns=['id'])    # 测试集特征 (无 Tm)



    # 随机选取部分特征（示例：50 个）
    if config["ISTEST"]:

        sample_len = 100
        features_train = train_df.drop(columns=['id', 'accident_risk']).iloc[:sample_len]  # 训练特征 (前 1000 条)
        target_train = train_df.iloc[:sample_len]['accident_risk']               # 训练目标
        features_test = test_df.drop(columns=['id'])                # 测试特征 (同样的特征列)




    # 3. 打印维度信息
    print("📊 数据拆分完成")
    print(f"训练集特征 features_train  shape   : {features_train.shape}")
    print(f"训练集目标   target_train  shape   : {target_train.shape}")
    print(f"测试集特征  features_test  shape   : {features_test.shape}")
    print(f"           features_train  类型    : {type(features_train)}")

    return features_train, target_train, features_test



### 特征生成

In [ ]:
def add_new_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    针对道路事故预测构造新的衍生特征
    输入:
        df : pd.DataFrame
    输出:
        df_new : pd.DataFrame，包含新增特征
    """
    df = df.copy()
    shape_before = df.shape  # 原始维度

    # 1. 交互项
    df['Speed_x_Curvature'] = df['speed_limit'] * df['curvature']
    df['Lanes_x_Speed']     = df['num_lanes'] * df['speed_limit']
    df['Accidents_x_Speed'] = df['num_reported_accidents'] * df['speed_limit']

    # 类别交互项（需要后续 OneHotEncoder）
    df['RoadType_Time']     = df['road_type'] + "_" + df['time_of_day']
    df['Lighting_Weather']  = df['lighting'] + "_" + df['weather']

    # 2. 非线性变换
    df['Log_Accidents']     = np.log1p(df['num_reported_accidents'])  # log(1+x)

    # 分箱（自定义阈值，可调节）
    df['Accident_Bins'] = pd.cut(df['num_reported_accidents'],
                                bins=[-1, 0, 2, 5, np.inf],
                                labels=['none', 'low', 'medium', 'high'])

    # 3. 比率特征
    df['Accidents_per_Lane']   = df['num_reported_accidents'] / (df['num_lanes'] + 1)
    df['Curvature_per_Lane']   = df['curvature'] / (df['num_lanes'] + 1)
    df['Speed_per_Lane']       = df['speed_limit'] / (df['num_lanes'] + 1)


    shape_after = df.shape  # 新的维度

    print(f"{shape_before[0]} × {shape_before[1]} --> {shape_after[0]} × {shape_after[1]}   新增 {shape_after[1] - shape_before[1]} 列")



    return df



### 手动编码 & 布尔转换

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def fit_ohe(train_df: pd.DataFrame):
    """
    在训练集上自动检测 object/category 列，并拟合 OneHotEncoder。
    返回 encoder 对象和类别列名。
    """
    # 检测类别列
    cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
    print(f"检测到{len(cat_cols)}类: {cat_cols}")

    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    ohe.fit(train_df[cat_cols])

    return ohe, cat_cols


def transform_ohe(df: pd.DataFrame, ohe: OneHotEncoder, cat_cols: list) -> pd.DataFrame:
    """
    使用已拟合的 OneHotEncoder 对 DataFrame 进行编码。
    自动处理 bool -> int，保持 train/test 一致。
    """
    df = df.copy()
    shape_before = df.shape

    # 布尔型转 int
    for col in df.select_dtypes('bool').columns:
        df[col] = df[col].astype(int)

    # OHE
    ohe_array = ohe.transform(df[cat_cols])
    ohe_cols = ohe.get_feature_names_out(cat_cols)
    ohe_df = pd.DataFrame(ohe_array, columns=ohe_cols, index=df.index)

    # 拼接
    df_enc = pd.concat([df.drop(columns=cat_cols), ohe_df], axis=1)

    shape_after = df_enc.shape
    print(f"{shape_before[0]} × {shape_before[1]} --> {shape_after[0]} × {shape_after[1]}   新增 {shape_after[1] - shape_before[1]} 列")

    return df_enc





### 特征生成

### PCA降维

In [ ]:
# 对数据降维

import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy import sparse

def fit_svd(train_df: pd.DataFrame, n_components: int = 100, random_state: int = 42) -> TruncatedSVD:
    """
    在训练集上拟合 TruncatedSVD，并返回拟合好的模型。
    """
    X_sparse = sparse.csr_matrix(train_df.values)
    svd = TruncatedSVD(n_components=n_components, random_state=random_state)
    svd.fit(X_sparse)
    explained_var = svd.explained_variance_ratio_.sum()
    print(f"✅ SVD 已拟合完成 (n_components={n_components})，训练集累计解释方差比: {explained_var:.2%}")
    return svd

def transform_svd(df: pd.DataFrame, svd: TruncatedSVD) -> pd.DataFrame:
    """
    使用拟合好的 SVD 对数据集进行降维。
    """
    shape_before = df.shape
    X_sparse = sparse.csr_matrix(df.values)
    X_reduced_array = svd.transform(X_sparse)

    reduced_df = pd.DataFrame(
        X_reduced_array,
        index=df.index,
        columns=[f"SVD_{i+1}" for i in range(X_reduced_array.shape[1])]
    )

    shape_after = reduced_df.shape
    print(f"{shape_before[0]} × {shape_before[1]} --> {shape_after[0]} × {shape_after[1]}")
    return reduced_df







# 交叉训练验证

In [ ]:
# Stratified K-Fold + XGBoost 进行训练验证，并保存实验结果
# ==============================================================
def run_kfold_xgb(features_train, target_train, features_test, config, DIRS, K_FOLDS=10, verbose=0):
    """
    使用 Stratified K-Fold + XGBoost 进行训练验证，并保存实验结果

    参数:
        features_train, target_train        : 训练集特征和标签
        features_test      : 测试集特征
        params      : XGBoost 最优参数 (dict)
        DIRS        : 保存结果的目录字典
        K_FOLDS     : 折数 (默认=5)
        verbose     : 是否打印详细信息
    """

    
        
    config["X shape"] = features_train.shape
    config["y shape"] = target_train.shape
    config["X_test shape"] = features_test.shape


    # ---------- 创建目录 ----------
    for _, path in DIRS.items():
        os.makedirs(path, exist_ok=True)


    time_str = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    history_DIR = os.path.join(DIRS['HISTORY'], time_str)
    os.makedirs(history_DIR, exist_ok=True)



    print("——" * 20)
    print(f"✅ 当前结果将保存到: {time_str}")


    # ---------- 定义交叉验证 ----------
    skfold = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
    yeo = PowerTransformer(method="yeo-johnson")                                # 定义 Yeo-Johnson 变换

    # ---------- 初始化存储 ----------
    oof_val = np.zeros(len(features_train))       # OOF 预测
    train_score, val_score = [], []  # 每折 LOSS
    test_pred = []                   # 每折 test 预测
    fold_records = []                # 保存每折信息
    all_importances = []             # 特征重要性
    elapsed_list = []                # 耗时记录



    # 循环每一折
    # ==============================================================

    for i, (train_idx, val_idx) in enumerate(skfold.split(features_train, pd.qcut(target_train, q=10).cat.codes), 1):

        # ----- 打印时间信息 -----
        start_now = datetime.now()
        start_str = start_now.strftime("%H:%M:%S")

        if elapsed_list:
            avg_time = np.mean(elapsed_list)
            est_end = start_now + timedelta(seconds=avg_time)

            # 每 5 个一组输出耗时
            parts = [f"{t:6.1f}s" for t in elapsed_list]
            grouped = [" ".join(parts[j:j+5]) for j in range(0, len(parts), 5)]
            elapsed_str = " /// ".join(grouped)

            print(
                f"🔄{i:2d}/{K_FOLDS} ST {start_str}"
                f" ET {est_end.strftime('%H:%M:%S')}"
                f" avg {avg_time:.1f}s"
                f" [{elapsed_str}]",
                end="\r", flush=True
            )
        else:
            print(f"🔄{i:2d}/{K_FOLDS} ST {start_str} ET (暂无历史数据)", end="\r", flush=True)



        # ----- 开始训练 -----
        t0 = time.time()

        # 1. 数据集划分
        x_train, x_val = features_train.iloc[train_idx], features_train.iloc[val_idx]
        y_train, y_val = target_train[train_idx], target_train[val_idx]

        # 2. Yeo-Johnson 变换
        y_train = yeo.fit_transform(y_train.values.reshape(-1, 1)).squeeze()
        y_val   = yeo.transform(y_val.values.reshape(-1, 1)).squeeze()


        # 3. 特征选择（轻量级 XGBoost）
        # 使用
        selector_model = xgb.XGBRegressor(**config["xgb_selector_model_params"])
        # selector_model = xgb.XGBRegressor(
        #     n_estimators   = 500,
        #     max_depth      = 6,
        #     learning_rate  = 0.05,
        #     random_state   = 2025,
        #     device         = "cpu",
        #     objective      = "reg:absoluteerror",
        #     tree_method    = "hist",
        #     verbosity      = 0
        # )
        
        

        selector_model.fit(x_train, y_train)

        selector = SelectFromModel(selector_model, prefit=True, threshold=config["selector_threshold"])
        selected_features = x_train.columns[selector.get_support()].tolist()
        if verbose > 0:
            print(f"✅ 选择的特征数量: {len(selected_features)}")


        # 4. 保留重要特征
        x_train_new = x_train[selected_features]
        x_val_new   = x_val[selected_features]
        x_test_new  = features_test[selected_features]







        # 5. 构造 LightGBM 数据集
        dtrain = lgb.Dataset(x_train_new, label=y_train)
        dval   = lgb.Dataset(x_val_new,   label=y_val)
        # dtest  = lgb.Dataset(x_test_new,  label=y_test)

        early_stop_rounds = 800 if config["lgb_train_model_params"]["learning_rate"] < 0.03 else 300
        use_early_stopping = (config["lgb_train_model_params"]["boosting_type"] != "dart")

        callbacks = []
        if use_early_stopping:
            callbacks.append(lgb.early_stopping(stopping_rounds=300))
        callbacks.append(lgb.log_evaluation(period=1000))

        # 6. LightGBM 训练
        lgb_model = lgb.train(
            params=config["lgb_train_model_params"],
            train_set=dtrain,
            valid_sets=[dtrain, dval],
            valid_names=["train", "valid"],
            num_boost_round=config["num_boost_round"],
        callbacks=callbacks
        )

        # # 保存模型
        # model_path = os.path.join(history_DIR, f"lgb_model_fold{i}.txt")
        # lgb_model.save_model(model_path)



        # 7. 获取特征重要性
        # imp_dict = xgb_model.get_score(importance_type="gain")
        # imp_df = pd.DataFrame(imp_dict.items(), columns=["Feature", "Importance"])
        # imp_df["Fold"] = i
        # all_importances.append(imp_df)


        # 8. 预测
        y_train_pred = lgb_model.predict(x_train_new, num_iteration=lgb_model.best_iteration)
        y_val_pred   = lgb_model.predict(x_val_new,   num_iteration=lgb_model.best_iteration)
        y_test_pred  = lgb_model.predict(x_test_new,  num_iteration=lgb_model.best_iteration)










        # 9. 逆变换
        y_train      = yeo.inverse_transform(y_train.reshape(-1, 1)).squeeze()
        y_val        = yeo.inverse_transform(y_val.reshape(-1, 1)).squeeze()
        y_train_pred = yeo.inverse_transform(y_train_pred.reshape(-1, 1)).squeeze()
        y_val_pred   = yeo.inverse_transform(y_val_pred.reshape(-1, 1)).squeeze()
        y_test_pred  = yeo.inverse_transform(y_test_pred.reshape(-1, 1)).squeeze()

        # 10. 计算 LOSS
        train_loss = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
        val_loss   = np.sqrt(np.mean((y_val   - y_val_pred) ** 2))


        if verbose > 0:
            print(f"Fold {i}: Train LOSS={train_loss:.4f}, Val LOSS={val_loss:.4f}，用时 {elapsed:.2f} 秒")


        # ----- 保存结果 -----
        train_score.append(train_loss)
        val_score.append(val_loss)
        oof_val[val_idx] = y_val_pred
        test_pred.append(y_test_pred)

        elapsed = time.time() - t0
        elapsed_list.append(elapsed)

        fold_records.append({
            "Fold": i,
            "Train_LOSS": train_loss,
            "Val_LOSS": val_loss,
            "Num_Features": len(selected_features),
            "Selected_Features": selected_features,
            "elapsed": elapsed
        })

    # 保存整体结果
    # ==============================================================
    if verbose > 0:
        print("\n")
        print(f"📊 Train LOSS 平均值 : {np.mean(train_score):.4f}")
        print(f"📊 Val   LOSS 平均值 : {np.mean(val_score):.4f}")
        print(f"📊 Train LOSS 标准差 : {np.std(train_score, ddof=0):.4f}")
        print(f"📊 Val   LOSS 标准差 : {np.std(val_score, ddof=0):.4f}")

    # 参数
    with open(os.path.join(history_DIR, "config.json"), "w", encoding="utf-8") as f:
        json.dump(config, f, indent=4, ensure_ascii=False)

    # 每折信息
    folds_df = pd.DataFrame(fold_records)
    folds_df.to_csv(os.path.join(history_DIR, "folds_info.csv"), index=False, encoding="utf-8-sig")


    # # 特征重要性
    # if all_importances:
    #     valid_imps = [df for df in all_importances if not df.empty]
    #     all_imp_df = pd.concat(valid_imps, axis=0) if valid_imps else pd.DataFrame(columns=["Feature", "Importance", "Fold"])
    # else:
    #     all_imp_df = pd.DataFrame(columns=["Feature", "Importance", "Fold"])
    # all_imp_df.to_csv(os.path.join(history_DIR, "feature_importance_all.csv"), index=False, encoding="utf-8-sig")


    # 测试集预测
    test_pred_array = np.vstack(test_pred).T
    test_pred_df = pd.DataFrame(test_pred_array, columns=[f"Fold_{j+1}" for j in range(test_pred_array.shape[1])])
    test_pred_df["Final_Pred"] = test_pred_df.mean(axis=1)
    test_pred_df.to_csv(os.path.join(history_DIR, "test_predictions.csv"), index=False, encoding="utf-8-sig")

    # 总结
    with open(os.path.join(history_DIR, "summary.txt"), "w", encoding="utf-8") as f:
        f.write(f"Train LOSS Mean : {np.mean(train_score):.4f}\n")
        f.write(f"Val   LOSS Mean : {np.mean(val_score):.4f}\n")
        f.write(f"Train LOSS Std  : {np.std(train_score, ddof=0):.4f}\n")
        f.write(f"Val   LOSS Std  : {np.std(val_score, ddof=0):.4f}\n")


    # 最终提交
    final_score = np.mean(val_score)
    submission = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "sample_submission.csv"))
    submission["accident_risk"] = test_pred_df["Final_Pred"]

    submission_path = os.path.join(history_DIR, f"sub_{time_str}_{final_score:.8f}.csv")
    submission.to_csv(submission_path, index=False)
    submission.to_csv(os.path.join(DIRS['SUBMISSION'], f"sub_{time_str}_{final_score:.8f}.csv"), index=False)

        
    config["time_str"] = time_str
    config["score"] = final_score


    # ---------- 返回结果 ----------
    return {
        "oof_val": oof_val,
        "train_score": train_score,
        "val_score": val_score,
        "test_pred": test_pred_df,
        "folds_info": folds_df,
        # "feature_importance": all_imp_df,
        "submission_path": submission_path,
        "time": time_str,
        "final_score": final_score,
        "config": config
    }


# 单次训练推导

In [ ]:
# 执行一次

# 准备流程---------------------------------------------------------------------------------------------------
# 打印当前config
print(config_to_str(config))

# Kaggle 提供的训练集和测试集
train_df = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "train.csv"))
test_df  = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "test.csv"))

# 数据拆分
print("数据拆分---------------------------")
features_train, target_train, features_test = prepare_features_and_target(train_df, test_df, config)


dataframe_info(features_train, n_sample = 6)


# 特征生成
print("特征生成---------------------------")
if config["use_feature_gen"]:
    features_train = add_new_features(features_train)
    features_test  = add_new_features(features_test)


dataframe_info(features_train, n_sample = 6)


# 手动编码 & 布尔转换
print("手动编码 & 布尔转换----------------")
# 在训练集上拟合
ohe, cat_cols = fit_ohe(features_train)
# 分别编码 train/test，保证一致
features_train = transform_ohe(features_train, ohe, cat_cols)
features_test  = transform_ohe(features_test,  ohe, cat_cols)


# 数据降维
print("数据降维---------------------------")
if config["use_pca"]:
    # 1. 在训练集上拟合
    svd = fit_svd(features_train, n_components = config["pca_components"], random_state=42)

    # 2. 分别对 train/test transform
    features_train_reduced = transform_svd(features_train, svd)
    features_test_reduced  = transform_svd(features_test, svd)

    shape_before_train = features_train.shape
    shape_before_test = features_test.shape

    features_train = pd.concat([features_train, features_train_reduced], axis=1)
    features_test = pd.concat([features_test, features_test_reduced], axis=1)

    shape_after_train = features_train.shape
    shape_after_test = features_test.shape

    print(f"Train: {shape_before_train[0]} × {shape_before_train[1]}  -->  {shape_after_train[0]} × {shape_after_train[1]}   新增 {shape_after_train[1] - shape_before_train[1]} 列")
    print(f"Test : {shape_before_test[0]} × {shape_before_test[1]}  -->  {shape_after_test[0]} × {shape_after_test[1]}   新增 {shape_after_test[1] - shape_before_test[1]} 列")



X, y, X_test = features_train, target_train, features_test
print("开始训练---------------------------")

# 准备流程---------------------------------------------------------------------------------------------------

results = run_kfold_xgb(X, y, X_test, config, DIRS, K_FOLDS = 10, verbose = 0)
config = results['config']
score = results['final_score']



print('\n',score)


# 提交 kaggle 平台测试

In [ ]:
# 根据 submission_time 定位文件路径 提交 kaggle 平台测试

import os
import itertools
import time
from kaggle.api.kaggle_api_extended import KaggleApi


def find_submission_file(submission_time, submission_dir):
    """
    在 submission_dir 下查找包含 submission_time 的文件
    一旦找到立刻返回完整路径；如果没找到则返回 None
    """
    for fname in os.listdir(submission_dir):
        if submission_time in fname:
            file_path = os.path.join(submission_dir, fname)
            print(f"✅ 找到目标文件: {fname}")
            return file_path
    
    print(f"⚠️ 未找到包含 {submission_time} 的文件")
    return None

def submit_and_get_score(file_path, competition_name, message="My submission"):
    """
    封装 Kaggle 提交并等待结果评分
    --------------------------------------
    file_path        : str  提交文件路径
    competition_name : str  Kaggle 比赛名称 (URL 最后一段)
    message          : str  提交备注
    """
    # 1. 配置 Kaggle API
    os.environ["KAGGLE_CONFIG_DIR"] = r"C:\Users\Admin\.kaggle"
    api = KaggleApi()
    api.authenticate()
    print("✅ Kaggle API 已经配置成功！")

    # 2. 提交文件
    api.competition_submit(
        file_name=file_path,
        competition=competition_name,
        message=message
    )
    print("✅ 提交完成！请等待评分...")

    # 3. 动态等待
    spinner = itertools.cycle(["|", "/", "-", "\\"])
    while True:
        submissions = api.competition_submissions(competition_name)
        latest = submissions[0]
        status_str = str(latest._status).lower()

        if "complete" in status_str and latest._public_score is not None:
            print("\n🎯 最终结果:")
            print(f"Public 分数 : {latest._public_score}")
            print(f"Private 分数: {latest._private_score}")
            print(f"提交 ID     : {latest._ref}")
            print(f"文件名      : {latest._file_name}")
            print(f"状态        : {latest._status}")
            print(f"提交时间    : {latest._date}")
            print(f"描述/备注   : {latest._description}")
            return latest

        if "pending" in status_str:
            spin_char = next(spinner)
            print(f"当前状态: {status_str} , 等待中 {spin_char}", end="\r", flush=True)
            time.sleep(0.2)  # 每 0.5 秒检查一次
            continue

        else:
            print(f"\n报错")
            print(f"submissions")
            
            break

        


### 不轻易运行，再三考虑

In [ ]:
# submission_time 提交
submission_time = "2025-10-24 01-23-41"
competition_name = kaggle_competition_name
message =  f"该提交文件的参数：\n{config_to_str(config)} "

target_file = find_submission_file(submission_time, DIRS['SUBMISSION'] )

# submit_and_get_score(target_file, competition_name, message)

# 参数优化

In [ ]:
# 实验配置单
base_config = {
    # 固定开关
    "ISTEST"            : False,

    "use_feature_gen"   : False,
    "use_pca"           : True,
    "pca_components"    : 10,



    "study_save_name"    : study_save_name,

    
    # 特征选择 XGBoost 参数
    "xgb_selector_model_params": {
        "n_estimators"  : 500,
        "max_depth"     : 6,
        "learning_rate" : 0.05,
        "random_state"  : 2025,
        "device"        : "cpu",
        "objective"     : "reg:squarederror",
        "tree_method"   : "hist",
        "verbosity"     : 0
    },

    "selector_threshold"  : "0*mean",   

    # # 训练设置
    # "xgb_train_model_params": {
    #     'max_depth'   : 6,
    #     'eta'         : 0.1,
    #     'tree_method' : 'hist',
    #     'eval_metric' : 'rmse',
    # },
    # "num_boost_round": 15000,


    # 训练设置（LightGBM）
    "lgb_train_model_params": {
        "objective": "regression",        # 回归任务
        "metric": "rmse",                 # 使用 RMSE 作为评估指标
        "boosting_type": "gbdt",          # 梯度提升树（默认）
        "learning_rate": 0.05,            # 学习率（原来 XGB 的 eta）
        "num_leaves": 31,                 # 控制树的复杂度
        "max_depth": -1,                  # -1 表示不限制深度
        "min_data_in_leaf": 20,           # 每个叶子最少样本数
        "feature_fraction": 0.9,          # 每次建树时使用的特征比例
        "bagging_fraction": 0.8,          # 每次建树时使用的数据比例
        "bagging_freq": 5,                # 每 5 次迭代进行一次 bagging
        "lambda_l1": 0.1,                 # L1 正则化
        "lambda_l2": 0.1,                 # L2 正则化
        "verbosity": -1                   # 不输出详细日志
    },
    "num_boost_round": 15000,             # 与 XGB 相同，用于控制最大迭代次数



}

In [ ]:
# 定义优化任务  加入标识符 host: hao-2   ip: 192.168.40.1

import copy
import contextlib
import io

def objective(trial):
    """
    Optuna 的目标函数 (Objective Function)
    每次 trial 会生成一组超参数，用于训练 XGBoost 模型，
    并返回交叉验证的平均 RMSE 作为优化目标。
    """




    # 1. 定义 超参数 搜索空间
    # 拷贝一份 config，避免全局污染
    config = copy.deepcopy(base_config)

    # 只修改需要优化的参数
    config["use_feature_gen"]   = trial.suggest_categorical("use_feature_gen", [True, False])
    config["use_pca"]           = trial.suggest_categorical("use_pca", [True, False])
    config["pca_components"] = trial.suggest_categorical("pca_components", [5, 10, 15, 20])

    # config["xgb_selector_model_params"]["random_state"] = trial.suggest_categorical("selector_random_state", [2025])
    # config["xgb_selector_model_params"]["device"]       = trial.suggest_categorical("selector_device", ["cpu", "cuda"])

    # config["selector_threshold"] = trial.suggest_categorical("selector_threshold", ["0*mean", "0.25*mean", "0.5*mean", "0.75*mean", "mean"])

    # config["xgb_train_model_params"]["max_depth"] = trial.suggest_int("train_max_depth", 3, 12)
    # config["xgb_train_model_params"]["eta"] = trial.suggest_float("train_eta", 0.01 , 0.3 , log=True)
    config["lgb_train_model_params"]["learning_rate"] = trial.suggest_float(
        "learning_rate", 0.005, 0.1, log=True
    )
    config["lgb_train_model_params"]["num_leaves"] = trial.suggest_int(
        "num_leaves", 16, 256
    )
    # config["lgb_train_model_params"]["max_depth"] = trial.suggest_int(
    #     "max_depth", -1, 15
    # )
    config["lgb_train_model_params"]["min_data_in_leaf"] = trial.suggest_int(
        "min_data_in_leaf", 10, 100
    )
    config["lgb_train_model_params"]["feature_fraction"] = trial.suggest_float(
        "feature_fraction", 0.6, 1.0
    )
    config["lgb_train_model_params"]["bagging_fraction"] = trial.suggest_float(
        "bagging_fraction", 0.6, 1.0
    )
    config["lgb_train_model_params"]["bagging_freq"] = trial.suggest_int(
        "bagging_freq", 1, 10
    )
    config["lgb_train_model_params"]["lambda_l1"] = trial.suggest_float(
        "lambda_l1", 1e-3, 10.0, log=True
    )
    config["lgb_train_model_params"]["lambda_l2"] = trial.suggest_float(
        "lambda_l2", 1e-3, 10.0, log=True
    )
    config["lgb_train_model_params"]["boosting_type"] = trial.suggest_categorical(
        "boosting_type", ["gbdt", "dart"]
    )







    # 创建一个黑洞缓冲区
    f = io.StringIO()
    with contextlib.redirect_stdout(f):
        None




        # 准备流程---------------------------------------------------------------------------------------------------
        # 打印当前config
        print(config_to_str(config))

        # Kaggle 提供的训练集和测试集
        train_df = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "train.csv"))
        test_df  = pd.read_csv(os.path.join(DIRS['DATA_DIR000'], "test.csv"))

        # 数据拆分
        print("数据拆分---------------------------")
        features_train, target_train, features_test = prepare_features_and_target(train_df, test_df, config)

        # 特征生成
        print("特征生成---------------------------")
        if config["use_feature_gen"]:
            features_train = add_new_features(features_train)
            features_test  = add_new_features(features_test)


        # 手动编码 & 布尔转换
        print("手动编码 & 布尔转换----------------")
        # 在训练集上拟合
        ohe, cat_cols = fit_ohe(features_train)
        # 分别编码 train/test，保证一致
        features_train = transform_ohe(features_train, ohe, cat_cols)
        features_test  = transform_ohe(features_test,  ohe, cat_cols)


        # 数据降维
        print("数据降维---------------------------")
        if config["use_pca"]:
            # 1. 在训练集上拟合
            svd = fit_svd(features_train, n_components = config["pca_components"], random_state=42)

            # 2. 分别对 train/test transform
            features_train_reduced = transform_svd(features_train, svd)
            features_test_reduced  = transform_svd(features_test, svd)

            shape_before_train = features_train.shape
            shape_before_test = features_test.shape

            features_train = pd.concat([features_train, features_train_reduced], axis=1)
            features_test = pd.concat([features_test, features_test_reduced], axis=1)

            shape_after_train = features_train.shape
            shape_after_test = features_test.shape

            print(f"Train: {shape_before_train[0]} × {shape_before_train[1]}  -->  {shape_after_train[0]} × {shape_after_train[1]}   新增 {shape_after_train[1] - shape_before_train[1]} 列")
            print(f"Test : {shape_before_test[0]} × {shape_before_test[1]}  -->  {shape_after_test[0]} × {shape_after_test[1]}   新增 {shape_after_test[1] - shape_before_test[1]} 列")



        X, y, X_test = features_train, target_train, features_test
        print("开始训练---------------------------")

        # 准备流程---------------------------------------------------------------------------------------------------





    results = run_kfold_xgb(X, y, X_test, config, DIRS, K_FOLDS = 10, verbose = 0)
    config = results['config']
    score = results['final_score']



    HOSTNAME = socket.gethostname()
    HOST_IP = socket.gethostbyname(HOSTNAME)
    trial.set_user_attr("host", HOSTNAME)        # 你自己定义主机 A/B
    trial.set_user_attr("ip", HOST_IP)        # 你自己定义角色 A/B

    # 4. 返回平均 LOSS
    return score




In [ ]:
# 开始优化

# 1. 定义 SQLite 数据库存储路径

storage_url = f"mysql+pymysql://{user}:{password}@{host}:3306/{database_name}"

STUDY_NAME = F"test_{study_save_name}" if base_config["ISTEST"] else study_save_name


study = optuna.create_study(
    study_name = STUDY_NAME,
    # study_name="ghsdjsrtjrswtjhwrt",
    storage=storage_url,
    load_if_exists=True
)

# 自动获取当前主机名\当前主机的 IP 地址
HOSTNAME = socket.gethostname()
HOST_IP = socket.gethostbyname(HOSTNAME)
print("主机名:", HOSTNAME," 主机 IP:", HOST_IP)
time.sleep(1)

# 5. 启动超参数搜索
print("🔎 开始超参数搜索...")
if base_config["ISTEST"]:
    study.optimize(objective, n_trials = 3)
else:
    study.optimize(objective, n_trials = 200)


# 6. 打印最优结果
print("\n✅ 训练完成！")
print(f"📊 已完成试验次数 : {len(study.trials)}")
print(f"🏆 最优试验编号   : {study.best_trial.number}")
print(f"📉 最优 LOSS       : {study.best_value}")
print(f"⚙️ 最优参数组合   : {study.best_trial.params}")


# 管理数据库信息

In [35]:
# 查询数据库详细数据


storage_url = f"mysql+pymysql://{user}:{password}@{host}:3306/{database_name}"

studies = optuna.study.get_all_study_summaries(storage=storage_url)

if not studies:
    print("❌ 当前数据库里无 study")
else:
    print("✅ 数据库中的 study 列表:")
    for s in studies:

        print("-", s.study_name)

        study = optuna.load_study(study_name=s.study_name, storage=storage_url)

        print("         Trials:")
        for trial in study.trials[:2]:  # 仅显示前 2 个 trial
            host = trial.user_attrs.get("host") or "unknown"
            ip = trial.user_attrs.get("ip") or "unknown"
            value = f"{trial.value:.10f}" if trial.value is not None else "None"

            print(
                f"    Trial {trial.number:4d}: "
                f"host={host:<16}, ip={ip:<15}, "
                f"value={value:<15}, params={trial.params}"
            )

        print("    总 trial 数量:", len(study.trials))
        print("=" * 100)


✅ 数据库中的 study 列表:
- XGBoost_model
         Trials:
    Trial    0: host=hao-2           , ip=192.168.40.1   , value=0.0616985820   , params={'use_feature_gen': False, 'use_pca': False, 'pca_components': 15, 'selector_threshold': 'mean', 'train_max_depth': 11, 'train_eta': 0.1343165303132418}
    Trial    1: host=hao-2           , ip=192.168.40.1   , value=0.0561359815   , params={'use_feature_gen': True, 'use_pca': False, 'pca_components': 5, 'selector_threshold': '0*mean', 'train_max_depth': 6, 'train_eta': 0.022101706430071286}
    总 trial 数量: 278
- LightGBM_model
         Trials:
    Trial    0: host=hao-2           , ip=192.168.40.1   , value=0.0562367515   , params={'use_feature_gen': True, 'use_pca': True, 'pca_components': 20, 'learning_rate': 0.05855433126993213, 'num_leaves': 242, 'min_data_in_leaf': 77, 'feature_fraction': 0.8454640426355255, 'bagging_fraction': 0.7469011972885259, 'bagging_freq': 7, 'lambda_l1': 6.693584206135924, 'lambda_l2': 0.2765348113468092, 'boosting_t

In [36]:
# 清理前：先查看数据库里当前有哪些 study 存在，以及每个 study 里有多少个 trial

storage_url = f"mysql+pymysql://{user}:{password}@{host}:3306/{database_name}"

studies = optuna.study.get_all_study_summaries(storage=storage_url)
print("现有 study：", [s.study_name for s in studies])

for s in studies:
    study = optuna.load_study(study_name=s.study_name, storage=storage_url)
    print(f"Study:   {s.study_name:30s}, Trials: {len(study.trials):4d}")

现有 study： ['XGBoost_model', 'LightGBM_model', 'test_XGBoost_model']
Study:   XGBoost_model                 , Trials:  278
Study:   LightGBM_model                , Trials:   10
Study:   test_XGBoost_model            , Trials:    9


In [33]:
# 清理中：删除指定 study
# 指定要删除的名称
to_delete = ["melting_point_study"]   # 可以写一个或多个

to_delete = [            ]

for s in studies:
    if s.study_name in to_delete:
        optuna.delete_study(study_name=s.study_name, storage=storage_url)
        print("已删除:", s.study_name)


In [34]:
# 清理后：再次检查
studies_after = optuna.study.get_all_study_summaries(storage=storage_url)
print("清理后 study：", [s.study_name for s in studies_after])


清理后 study： ['XGBoost_model', 'LightGBM_model', 'test_XGBoost_model']
